source .venv/bin/activate

LINEAR REGRESSION

Import Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error, r2_score

We load the log-transformed dataset created earlier, which is suitable for linear models.

In [2]:
df = pd.read_csv('../data/clean_log.csv')
df.head()

,price,mileage,tax,mpg,engineSize,car_age,model_freq,brand_BMW,brand_Mercedes,brand_VW,transmission_Manual,transmission_Semi-Auto,fuelType_Hybrid,fuelType_Other,fuelType_Petrol,log_price
0,11995.0,22606,30,57.6,1.4,9,709,False,False,False,True,False,False,False,True,9.392329
1,6000.0,129680,30,64.2,2.0,12,734,False,False,False,True,False,False,False,False,8.699681
2,13920.0,47576,20,67.3,2.0,9,2532,False,False,True,True,False,False,False,False,9.541154
3,10600.0,30983,30,53.3,1.4,10,2532,False,False,True,True,False,False,False,True,9.268704
4,21699.0,5229,145,43.5,1.5,6,2532,False,False,True,False,True,False,False,True,9.985068


Separate features and target variable, then split the data into training and validation sets.

In [3]:
X = df.drop(columns=['price', 'log_price'])
y = df['log_price']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

Ensure that all features are numeric. This is a safeguard step.

In [4]:
X = X.astype(float)  # force all features to be numeric

We standardize the features using StandardScaler to ensure zero mean and unit variance, which is important for linear models.

In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

Fit the linear regression model on scaled training data and make predictions.

In [7]:
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)
y_pred_log = lr.predict(X_val_scaled)

Convert predicted and true values back from log scale and compute RMSE and R^2.

In [8]:
y_val_raw = np.expm1(y_val)
y_pred_raw = np.expm1(y_pred_log)

rmse = root_mean_squared_error(y_val_raw, y_pred_raw)
r2 = r2_score(y_val_raw, y_pred_raw)

print(f"RMSE: ${rmse:.2f}")
print(f"R² Score: {r2:.3f}")

RMSE: $3621.28
R² Score: 0.844


Linear Regression (log target):
  RMSE = $3,621.28
  R²   = 0.844

To save the full pipeline, reload the full dataset and refit the model.

In [11]:
df_log = pd.read_csv('../data/clean_log.csv')

Create and save a full pipeline including standardization and regression for future predictions.

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from joblib import dump

# Full pipeline with scaler
lr_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', LinearRegression())
])

# Refit on full log-transformed data
X = df_log.drop(columns=['price', 'log_price'])
y = df_log['log_price']

lr_pipeline.fit(X, y)

# Save the pipeline
dump(lr_pipeline, '../Results/linear_model.joblib')

['../Results/linear_model.joblib']

### ✅ Summary
- Model: Linear Regression (with log-transformed target)
- Preprocessing: StandardScaler
- RMSE: $3621.28
- R² Score: 0.844
- Saved model: `linear_model.joblib`